In [1]:
import os
import sys
sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(), '../../')))

In [2]:
import numpy

In [3]:
import matplotlib.pyplot as plt

In [4]:
from spatialpy import (
    Model,
    Domain,
    Species,
    Parameter,
    Reaction,
    Geometry
)

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
class Edge1(Geometry):
    def __init__(self, xmax):
        self.xmax = xmax

    def inside(self, point, on_boundary):
        return abs(point[0] - self.xmax) < 0.05

In [7]:
class Edge2(Geometry):
    def __init__(self, xmin):
        self.xmin = xmin

    def inside(self, point, on_boundary):
        return abs(point[0] - self.xmin) < 0.05

In [8]:
class Middle(Geometry):
    def __init__(self, xmin):
        self.xmin = xmin

    def inside(self, point, on_boundary):
        return abs(point[0] - self.xmin) >= 0.05

In [9]:
class cylinderDemo3D(Model):
    # Define type IDs as constants of the Model
    MIDDLE = Middle.__name__
    EDGE1 = Edge1.__name__
    EDGE2 = Edge2.__name__
    
    def __init__(self, model_name="3D Cylinder Demo"):
        Model.__init__(self, model_name)

        # Set domain bounds
        xmin = -5
        xmax = 5
        
        # System constants
        D_const = 0.1

        """
        Create a domain from a FEniCS/dolfin style XML mesh file
        
        - filename: Name of the mesh file.
        """
        domain = Domain.read_xml_mesh(filename='../Domain_Files/cylinder.xml')
        # Add type IDs to the particles
        """
        Update the prpoerties of particles within a geometric shape.

        - geometry_ivar: an instance of a 'spatialpy.Geometry' subclass.  The 'inside()' method
                         of this object will be used to add a single point particles.
        - type_id: Particle type ID of particle to be created.
        - vol: Default volume of particle to be added.
        - mass: Default mass of particle to be added.
        - nu: Default viscosity of particle to be created.
        - c: Default artificial speed of sound of particle to be created.
        - rho: Default density of particle to be created
        - fixed: True if particle is spatially fixed, else False.
        """
        domain.set_properties(Middle(xmin=xmin), self.MIDDLE)
        domain.set_properties(Edge1(xmax=xmax), self.EDGE1)
        domain.set_properties(Edge2(xmin=xmin), self.EDGE2)
        # Add the Domain to the Model
        self.add_domain(domain)

        # Define Species
        A = Species(name="A", diffusion_coefficient=D_const, restrict_to=[self.MIDDLE, self.EDGE1])
        B = Species(name="B", diffusion_coefficient=D_const, restrict_to=[self.MIDDLE, self.EDGE2])
        
        # Add the Species to the Model.
        self.add_species([A, B])

        # Define Parameters
        vol = self.domain.get_vol()
        type_id = self.domain.type_id
        left = numpy.sum(vol[type_id == self.domain.get_type_def(self.EDGE1)])
        right = numpy.sum(vol[type_id == self.domain.get_type_def(self.EDGE2)])
        
        k_react = Parameter(name="k_react", expression=1.0)
        k_creat1 = Parameter(name="k_creat1", expression=100 / left)
        k_creat2 = Parameter(name="k_creat2", expression=100 / right)
        
        # Add the Parameters to the Model.
        self.add_parameter([k_react, k_creat1, k_creat2])
        
        # Define Reactions
        R1 = Reaction(
            reactants={},
            products={"A":1},
            rate="k_creat1",
            restrict_to=self.EDGE1
        )
        
        R2 = Reaction(
            reactants={},
            products={"B":1},
            rate="k_creat2",
            restrict_to=self.EDGE2
        )
        
        R3 = Reaction(
            reactants={"A":1, "B":1},
            products={},
            rate="k_react"
        )
        
        # Add the Reactions to the Model.
        self.add_reaction([R1, R2, R3])
        
        """
        Use NumPy to set the timespan of the Model. Reaction diffusion examples may require a small
        timespan_size (around 1e-3) to generate accurate deterministic results.
        """
        self.timespan(numpy.linspace(0, 500, 501), timestep_size=1e-5)

In [10]:
model = cylinderDemo3D()

In [ ]:
print(f"Number of Timesteps: {model.num_timesteps}")
print(f"Output Frequency: {model.output_freq}")
print(f"Timestep Size: {model.timestep_size}")
%time results = model.run()

Number of Timesteps: 50000000
Output Frequency: 99999.99999999999
Timestep Size: 1e-05


In [ ]:
model.timestep_size = 1
model.timespan(range(500))
print(f"Number of Timesteps: {model.num_timesteps}")
print(f"Output Frequency: {model.output_freq}")
print(f"Timestep Size: {model.timestep_size}")
%time results2 = model.run()

# Visualization

In [ ]:
def check_accuracy(results, deterministic=False):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 4.5))

    tspan = results.get_timespan()
    A_sum = numpy.sum(results.get_species("A", deterministic=deterministic), axis=1)
    B_sum = numpy.sum(results.get_species("B", deterministic=deterministic), axis=1)

    ax1.plot(tspan, A_sum, '-r', label="A")
    ax1.plot(tspan, B_sum, '-b', label="B")
    ax1.set_xlabel("Time", fontsize=14)
    ax1.tick_params(axis="x", labelsize=12)
    ax1.tick_params(axis="y", labelsize=12)
    _ = ax1.legend(loc='best')

    x_vals = results.model.domain.coordinates()[:, 0]
    A_vals = numpy.sum(results.get_species("A", deterministic=deterministic), axis=0)
    B_vals = numpy.sum(results.get_species("B", deterministic=deterministic), axis=0)

    ax2.plot(x_vals, A_vals, '.r')
    ax2.plot(x_vals, B_vals, '.b')
    ax2.set_xlabel("X-Coordinate", fontsize=14)
    ax2.tick_params(axis="x", labelsize=12)
    ax2.tick_params(axis="y", labelsize=12)
    _ = ax2.legend(['A', 'B'], loc='best')

In [ ]:
def analyze_data(results, t_ndx=0):
    _, data = results.read_step(step_num=t_ndx)
    
    import plotly.graph_objects as go

    concentration_data = data["D[A]"] / (data['mass'] / data['rho'])

    fig = go.Figure()
    fig.add_trace(go.Histogram(x=data["C[A]"]))
    fig.add_trace(go.Histogram(x=data["D[A]"]))
    fig.add_trace(go.Histogram(x=concentration_data))

    fig.update_layout(
        xaxis_title_text='Value', # xaxis label
        yaxis_title_text='Count', # yaxis label
        bargap=0.2, # gap between bars of adjacent location coordinates
        bargroupgap=0.1 # gap between bars of the same location coordinates
    )
    fig.show()

### Time Step Size = 1e-3

In [ ]:
check_accuracy(results)

In [ ]:
check_accuracy(results, deterministic=True)

In [ ]:
analyze_data(results=results, t_ndx=200)

In [ ]:
results.plot_species('A', t_ndx=200, width=None, height=None, title="Raw Stochastic Results")
results.plot_species('A', t_ndx=200, concentration=True, width=None, height=None, title="Stochastic Results")
results.plot_species('A', t_ndx=200, deterministic=True, width=None, height=None, title="Deterministic Results")

### Time Step Size = 1

In [ ]:
check_accuracy(results2)

In [ ]:
check_accuracy(results2, deterministic=True)

In [ ]:
analyze_data(results=results2, t_ndx=200)

In [ ]:
results2.plot_species('A', t_ndx=200, width=None, height=None, title="Raw Stochastic Results")
results2.plot_species('A', t_ndx=200, concentration=True, width=None, height=None, title="Stochastic Results")
results2.plot_species('A', t_ndx=200, deterministic=True, width=None, height=None, title="Deterministic Results")